In [1]:
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
import os

In [2]:
# normalize
def MyNormalize(x):
    x.fillna(0, inplace=True)
    mean_ = x.mean(axis=0)
    x -= mean_
    max_ = x.max(axis=0)
    x /= (max_+1e-5)
    
    return x

In [3]:
def feature_extract(x):
    # multi channel -> one channel
    # rms: root mean squared
    # rms_res = librosa.feature.rms(x.T)
    rms_res = []
    # spectral_centroid
    # Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins, from which the mean (centroid) is extracted per frame.
    sc_res = []
    # spectral_bandwidth
    # (sum_k S[k, t] * (freq[k, t] - centroid[t])**p)**(1/p)
    sb_res = []
    # zero_crossing_rate
    # 过零率（zero crossing rate）是一个信号符号变化的比率，即，在每帧中，语音信号从正变为负或从负变为正的次数
    zc_res = []
    # mel frequency cepstral coefficient
    mfcc_res = []
    # poly_features
    # Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram
    pf_res = []
    # spectral_contrast
    # Each frame of a spectrogram S is divided into sub-bands. For each sub-band, the energy contrast is estimated by comparing the mean energy in the top quantile (peak energy) to that of the bottom quantile (valley energy). High contrast values generally correspond to clear, narrow-band signals, while low contrast values correspond to broad-band noise.
    scon_res = []
    # chroma_stft after applying stft (or not)
    for coli in x:
        rms_tmp = librosa.feature.rms(x[coli].values)
        rms_res.append(rms_tmp)
        sc_tmp = librosa.feature.spectral_centroid(y=x[coli].values, sr=100)
        sc_res.append(sc_tmp)
        sb_tmp = librosa.feature.spectral_bandwidth(y=x[coli].values, sr=100)
        sb_res.append(sb_tmp)
        zc_tmp = librosa.feature.zero_crossing_rate(x[coli].values)
        zc_res.append(zc_tmp)
        scon_tmp = librosa.feature.spectral_contrast(y=x[coli].values, sr=100, fmin=1)
        scon_res.append(scon_tmp)
        pf_tmp = librosa.feature.poly_features(y=x[coli].values, sr=100, order=2)
        pf_res.append(pf_tmp)
        mfcc_tmp = librosa.feature.mfcc(y=x[coli].values, sr=100)
        mfcc_res.append(mfcc_tmp)
        
    final_res = []
    for res in [rms_res, sc_res, sb_res, zc_res, scon_res, pf_res, mfcc_res]:
        final_res.extend(res)
    
    return np.concatenate(final_res)

In [4]:
!mkdir ./train ./test

In [5]:
# train
def output(ppath, opath):
    for path in tqdm(os.listdir(ppath)):
        filepath = os.path.join(ppath, path)
        data = pd.read_csv(filepath)
        data = MyNormalize(data)
        res = feature_extract(data)
        outputname = os.path.join(opath, path[:-4]+'.npz')
        np.savez(outputname, res)

In [6]:
output(r'../input/predict-volcanic-eruptions-ingv-oe/train', r'./train')

100%|██████████| 4431/4431 [1:02:35<00:00,  1.18it/s]


In [7]:
output(r'../input/predict-volcanic-eruptions-ingv-oe/test', r'./test')

100%|██████████| 4520/4520 [1:02:50<00:00,  1.20it/s]
